## RAG 데이터 확보
- Allganize의 RAG 데이터 중 금융 도메인 데이터 200개 활용

In [ ]:
# https://huggingface.co/datasets/allganize/rag-ko

from huggingface_hub import snapshot_download

snapshot_download(
    repo_id='allganize/rag-ko',
    repo_type='dataset',
    local_dir='./res/rag-ko',
    local_dir_use_symlinks=False
)

print()

### 데이터 설명
- 질문과 정답이 있고, 답변에 필요한 정보(Context)가 질문 별로 3개씩 존재
  - 3개의 정보 중 하나가 실제로 필요한 정보이고, 나머지 2개는 필요하지 않은 정보
- Column 별 설명
  - system: 시스템 프롬프트 (Context 3개가 여기에 있음)
  - humman: 질문
  - answer_context_summary: 실제로 필요한 Context
  - answer_position: 3개의 Context 중 몇 번째가 실제로 필요한 Context인지
  - answer: 정답

In [ ]:
import pandas as pd


df = pd.read_parquet('./res/rag-ko/data/test-00000-of-00001.parquet')
df

In [ ]:
print(df.iloc[0]['system'])

In [ ]:
print(df.iloc[0]['answer_context_summary'])

시스템 프롬프트에서 3개의 Context를 파싱

In [ ]:
import re

def extract_contexts(system_prompt):
    context_match = re.search(r'CONTEXT="""\n(.*?)"""', system_prompt, re.DOTALL)
    if not context_match:
        return []

    context_text = context_match.group(1)

    contexts = re.findall(r'\(context \d+\)=(.*?)(?=\n\(context \d+\)=|\Z)', context_text, re.DOTALL)

    cleaned_contexts = []
    for context in contexts:
        lines = context.split('\n')
        cleaned_lines = [line for line in lines if not line.strip().startswith('Title:')]
        cleaned_context = '\n'.join(cleaned_lines).strip()
        cleaned_contexts.append(cleaned_context)

    return cleaned_contexts

system_prompt = df.iloc[0]['system']

extracted_contexts = extract_contexts(system_prompt)

for i, context in enumerate(extracted_contexts, 1):
    print(f"Context {i}:")
    print(context)
    print("-" * 50)

In [ ]:
extracted_contexts[2] == df.iloc[0]['answer_context_summary']

In [ ]:
contexts = []
for i in range(len(df)):
    system_prompt = df.iloc[i]['system']
    extracted_contexts = extract_contexts(system_prompt)
    contexts.append(extracted_contexts)

In [ ]:
questions = [df.iloc[i]['human'] for i in range(len(df))]
contexts_answers_idxs = [df.iloc[i]['answer_position'] - 1 for i in range(len(df))]
contexts_answers = [df.iloc[i]['answer_context_summary'] for i in range(len(df))]
answers = [df.iloc[i]['answer'] for i in range(len(df))]

In [ ]:
import pickle

rag_data = {
    'questions': questions,
    'contexts': contexts,
    'contexts_answer_idx': contexts_answers_idxs,
    'contexts_answers': contexts_answers,
    'answers': answers
}

with open('./res/rag_data.pkl', 'wb') as f:
    pickle.dump(rag_data, f)

### Accuracy
- 정답이 있는 상황이라 가장 정확한 평가
- Reference 또는 정답이 있을 때만 사용 가능
  - 실제로는 없는 경우가 대부분

In [ ]:
from utils import get_embedding, cosine_similarity


embed_q = get_embedding(questions[0], model='text-embedding-3-large')
embed_c0 = get_embedding(contexts[0][0], model='text-embedding-3-large')
embed_c1 = get_embedding(contexts[0][1], model='text-embedding-3-large')
embed_c2 = get_embedding(contexts[0][2], model='text-embedding-3-large')

In [ ]:
cosine_similarity(embed_q, embed_c0)

In [ ]:
cosine_similarity(embed_q, embed_c1)

In [ ]:
cosine_similarity(embed_q, embed_c2)

### RAGAS
- 정답이 없는 경우에도 사용이 가능한 평가 라이브러리
- 기본으로 제공하는 평가용 System Prompt는 영문 기반이라 엄청 정확하진 않은 편

In [ ]:
from datasets import Dataset 
import os
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness, context_relevancy, context_recall, context_precision


data_samples = {
    'question': [questions[0]],
    'answer': ['hi'],
    'contexts' : [contexts[1]],
    'ground_truth': [df.iloc[0]['answer']]
}

dataset = Dataset.from_dict(data_samples)

score = evaluate(dataset,metrics=[context_recall, context_precision])
score.to_pandas()

In [ ]:
contexts[0]

In [ ]:
df.iloc[0]['answer']

In [ ]:
contexts[i][0]

In [ ]:
contexts[i][1]

In [ ]:
contexts[i][2]

In [ ]:
df.iloc[i]['answer']

In [ ]:
for i in range(10):
    data_samples = {
        'question': [questions[i]],
        'answer': ['hi'],
        'contexts' : [contexts[i]],
        'ground_truth': [df.iloc[i]['answer']]
    }

    dataset = Dataset.from_dict(data_samples)

    score = evaluate(dataset,metrics=[context_recall, context_precision])
    print(score)